# Equities Pair Trading with Kalman Filter

In [1]:
using Feather, DataFrames, CSV, DelimitedFiles, Statistics

## Data Preprocessing

#### The Financials Sector

In [2]:
fin = CSV.read("financial_stocks.csv")
fin_ticker_raw = fin.Ticker
size(fin_ticker_raw,1)

1215

In [3]:
df = Feather.read("bar_data_1D/allDaily-A.feather")
size(df)

(359, 6)

In [4]:
head(df)

,Date,Open,High,Low,Close,Volume
,String,Float64,Float64,Float64,Float64,Int32
1,2017-06-26,59.93,59.96,59.14,59.24,1467388
2,2017-06-27,59.13,59.57,58.84,58.88,1016895
3,2017-06-28,59.27,59.7,59.2,59.4,1152733
4,2017-06-29,59.35,59.63,58.49,58.8,1323217
5,2017-06-30,59.0,59.57,58.84,59.31,1507308
6,2017-07-03,59.59,59.98,59.12,59.13,1159308


In [5]:
df.Date[1], df.Date[end]

("2017-06-26", "2018-11-26")

In [6]:
fin_data_raw = zeros((size(fin_ticker_raw,1), size(df,1)))
fin_vol_raw = zeros(Int32, (size(fin_ticker_raw,1), size(df,1)))

1215×359 Array{Int32,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0 

In [7]:
for i in 1:size(fin_ticker_raw,1)
    try
        filename = string("bar_data_1D/allDaily-", fin_ticker_raw[i], ".feather")
        tmpdf = Feather.read(filename)
        fin_data_raw[i,:] = [tmpdf.Close[j] for j in 1:size(tmpdf,1)]
        fin_vol_raw[i,:] = [tmpdf.Volume[j] for j in 1:size(tmpdf,1)]
    catch y
        #@warn("Exception:", y)
    end
end

In [8]:
fin_data_raw

1215×359 Array{Float64,2}:
   0.0     0.0     0.0     0.0     0.0   …    0.0     0.0     0.0     0.0 
   0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0 
   0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0 
  23.6    23.25   23.35   23.3    23.65      29.35   29.88   29.99   29.92
  46.35   47.05   48.15   48.75   48.2       42.46   42.46   42.49   43.16
  13.67   13.71   13.37   13.5    13.59  …    6.32    6.43    6.46    6.37
  33.05   32.85   34.4    34.05   34.0       42.11   42.81   43.22   42.51
   0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0 
   0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0 
   0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0 
   0.0     0.0     0.0     0.0     0.0   …    0.0     0.0     0.0     0.0 
  29.85   29.7    30.0    30.55   30.5       37.56   36.52   35.9    36.38
   0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0

In [9]:
cnt = 0
for i in 1:size(fin_data_raw,1)
    if fin_data_raw[i,1] != 0
        cnt += 1
    end
end
cnt

465

In [10]:
fin_data = zeros((cnt, size(df,1)))
fin_vol = zeros(Int32, (cnt, size(df,1)))
fin_ticker = Array{String}(undef, cnt)
n = 0
for i in 1:size(fin_data_raw,1)
    if fin_data_raw[i,1] != 0
        n += 1
        fin_data[n,:] = [fin_data_raw[i,k] for k in 1:size(fin_data_raw, 2)]
        fin_vol[n,:] = [fin_vol_raw[i,k] for k in 1:size(fin_vol_raw, 2)]
        fin_ticker[n] = fin_ticker_raw[i]
    end
end

In [11]:
@assert(n == cnt)

Filter valid data

In [12]:
open("finticker.csv", "w") do f
    writedlm(f, fin_ticker, ',')
end

open("findata.csv", "w") do f
    writedlm(f, fin_data, ',')
end

open("finvol.csv", "w") do f
    writedlm(f, fin_vol, ',')
end

#### The Technology Sector

In [13]:
tech = CSV.read("technology_stocks.csv")
tech_ticker_raw = tech.Ticker
size(tech_ticker_raw,1)

327

In [14]:
tech_data_raw = zeros((size(tech_ticker_raw,1), size(df,1)))
tech_vol_raw = zeros(Int32, (size(tech_ticker_raw,1), size(df,1)))

327×359 Array{Int32,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  

In [15]:
for i in 1:size(tech_ticker_raw,1)
    try
        filename = string("bar_data_1D/allDaily-", tech_ticker_raw[i], ".feather")
        tmpdf = Feather.read(filename)
        tech_data_raw[i,:] = [tmpdf.Close[j] for j in 1:size(tmpdf,1)]
        tech_vol_raw[i,:] = [tmpdf.Volume[j] for j in 1:size(tmpdf,1)]
    catch y
        #@warn("Exception:", y)
    end
end

In [16]:
cnt = 0
for i in 1:size(tech_data_raw,1)
    if tech_data_raw[i,1] != 0
        cnt += 1
    end
end
cnt

132

In [17]:
tech_data = zeros((cnt, size(df,1)))
tech_vol = zeros(Int32, (cnt, size(df,1)))
tech_ticker = Array{String}(undef, cnt)
n = 0
for i in 1:size(tech_data_raw,1)
    if tech_data_raw[i,1] != 0
        n += 1
        tech_data[n,:] = [tech_data_raw[i,k] for k in 1:size(tech_data_raw, 2)]
        tech_vol[n,:] = [tech_vol_raw[i,k] for k in 1:size(tech_vol_raw, 2)]
        tech_ticker[n] = tech_ticker_raw[i]
    end
end
@assert(n == cnt)

In [18]:
open("techticker.csv", "w") do f
    writedlm(f, tech_ticker, ',')
end

open("techdata.csv", "w") do f
    writedlm(f, tech_data, ',')
end

open("techvol.csv", "w") do f
    writedlm(f, tech_vol, ',')
end

In [19]:
DelimitedFiles.readdlm("techdata.csv", ',')

132×359 Array{Float64,2}:
 145.82  143.73  145.83  143.68  144.02  …  176.98  176.78  172.29  174.62
 122.34  122.19  123.74  122.99  123.68     158.9   156.98  156.93  158.4 
  80.57   78.14   79.2    77.74   77.8       89.02   89.68   88.64   90.07
  20.0    20.05   20.5    20.4    20.65      12.55   12.94   12.87   12.73
  43.25   41.89   42.69   41.44   41.31      35.77   35.19   35.05   36.01
  26.69   26.54   26.99   26.64   27.22  …   44.33   46.25   46.7    46.94
  10.81   10.54   10.67   10.21   10.29      10.39   10.84   11.01   10.93
  74.76   74.23   75.11   73.76   73.82      84.17   84.87   84.98   84.95
 133.78  132.37  134.99  129.6   130.31     161.33  163.91  163.66  167.43
  60.22   58.29   58.81   57.58   57.57      49.26   50.85   50.05   50.91
 242.75  235.33  241.05  234.04  233.05  …  227.71  230.0   229.9   235.31
  38.94   38.63   39.2    38.98   38.88      42.04   42.76   42.77   43.36
  16.56   16.16   16.55   16.35   16.34      15.77   16.02   16.17   16.06

## Data Filtration
The data filtration is based on stock beta and market cap. We need firstly figure out the beta and average daily volume (ADV) for each stock.<br>
First 200 days of data are used for this filtering process.

### Beta Calculation

In [20]:
fin_data = DelimitedFiles.readdlm("findata.csv", ',')
tech_data = DelimitedFiles.readdlm("techdata.csv", ',')

132×359 Array{Float64,2}:
 145.82  143.73  145.83  143.68  144.02  …  176.98  176.78  172.29  174.62
 122.34  122.19  123.74  122.99  123.68     158.9   156.98  156.93  158.4 
  80.57   78.14   79.2    77.74   77.8       89.02   89.68   88.64   90.07
  20.0    20.05   20.5    20.4    20.65      12.55   12.94   12.87   12.73
  43.25   41.89   42.69   41.44   41.31      35.77   35.19   35.05   36.01
  26.69   26.54   26.99   26.64   27.22  …   44.33   46.25   46.7    46.94
  10.81   10.54   10.67   10.21   10.29      10.39   10.84   11.01   10.93
  74.76   74.23   75.11   73.76   73.82      84.17   84.87   84.98   84.95
 133.78  132.37  134.99  129.6   130.31     161.33  163.91  163.66  167.43
  60.22   58.29   58.81   57.58   57.57      49.26   50.85   50.05   50.91
 242.75  235.33  241.05  234.04  233.05  …  227.71  230.0   229.9   235.31
  38.94   38.63   39.2    38.98   38.88      42.04   42.76   42.77   43.36
  16.56   16.16   16.55   16.35   16.34      15.77   16.02   16.17   16.06

Calculate daily returns

In [21]:
fin_ret = fin_data[:,2:end] ./ fin_data[:,1:end-1] .- 1
tech_ret = tech_data[:,2:end] ./ tech_data[:,1:end-1] .- 1

132×358 Array{Float64,2}:
 -0.0143327    0.0146107   -0.0147432    …  -0.0253988     0.0135237  
 -0.00122609   0.0126852   -0.0060611       -0.000318512   0.00936723 
 -0.0301601    0.0135654   -0.0184343       -0.0115968     0.0161327  
  0.0025       0.0224439   -0.00487805      -0.00540958   -0.010878   
 -0.0314451    0.0190976   -0.0292809       -0.0039784     0.0273894  
 -0.00562008   0.0169555   -0.0129678    …   0.00972973    0.00513919 
 -0.0249769    0.012334    -0.0431115        0.0156827    -0.00726612 
 -0.00708935   0.011855    -0.0179736        0.0012961    -0.000353024
 -0.0105397    0.019793    -0.0399289       -0.00152523    0.0230356  
 -0.0320492    0.00892091  -0.0209148       -0.0157325     0.0171828  
 -0.0305664    0.0243063   -0.0290811    …  -0.000434783   0.023532   
 -0.00796097   0.0147554   -0.00561224       0.000233863   0.0137947  
 -0.0241546    0.0241337   -0.0120846        0.0093633    -0.00680272 
  ⋮                                      ⋱         

In [22]:
spy = CSV.read("SPY.csv")
spy_data = spy.Adj
spy_ret = spy_data[2:end] ./ spy_data[1:end-1] .- 1

xlf = CSV.read("XLF.csv")
xlf_data = xlf.Adj
xlf_ret = xlf_data[2:end] ./ xlf_data[1:end-1] .- 1

xlk = CSV.read("XLK.csv")
xlk_data = xlk.Adj
xlk_ret = xlk_data[2:end] ./ xlk_data[1:end-1] .- 1

358-element Array{Float64,1}:
 -0.01642846955044497  
  0.01252721878795926  
 -0.01793082269815094  
 -0.0009129112977402931
 -0.006944361415459954 
  0.009937389199596192 
 -0.009110795928583326 
  0.011585071629201416 
  0.007271427552016263 
  0.0021656916931596104
  0.012965923064585994 
  0.0019556664828563886
  0.00887149418455735  
  ⋮                    
 -0.000843483797700828 
 -0.01730693682285067  
 -0.03522335539652666  
  0.0011873256687737754
 -0.012007071034791927 
  0.02505623330663953  
 -0.0016100848946135615
 -0.0381175493053445   
 -0.021490687067662106 
  0.007320888192565045 
 -0.009123179499711975 
  0.02262791752097648  

Compute beta
$$\beta_s = \frac{Cov(r_s\cdot r_m)}{Var(r_m)}$$

In [23]:
fin_beta_spy = [Statistics.cov(fin_ret[i,1:200], spy_ret[1:200]) / Statistics.var(spy_ret[1:200]) for i in 1:size(fin_ret,1)]
fin_beta_xlf = [Statistics.cov(fin_ret[i,1:200], xlf_ret[1:200]) / Statistics.var(xlf_ret[1:200]) for i in 1:size(fin_ret,1)]
tech_beta_spy = [Statistics.cov(tech_ret[i,1:200], spy_ret[1:200]) / Statistics.var(spy_ret[1:200]) for i in 1:size(tech_ret,1)]
tech_beta_xlk = [Statistics.cov(tech_ret[i,1:200], xlk_ret[1:200]) / Statistics.var(xlk_ret[1:200]) for i in 1:size(tech_ret,1)]

132-element Array{Float64,1}:
  0.9840402705323659 
  0.7954807137863318 
  1.0140488753985606 
  0.4554084828357748 
  1.6527798286201585 
  0.5534114697994453 
  0.6276404766211574 
  0.7100776972104891 
  1.192695765830311  
  1.267669032300593  
  1.0472403395289165 
  0.6765664354177291 
  0.7873835443099566 
  ⋮                  
  0.9540394876244636 
  1.1023035058111603 
  0.512740415992383  
  0.46347026411786846
  0.8107518704732612 
  1.0481545032291388 
  0.6539003117258995 
  0.4372294533449464 
  0.4341276957185432 
 -0.07384774417942211
  1.1373079954357863 
  0.5463467002994926 

### ADV Calculation

In [24]:
fin_vol = DelimitedFiles.readdlm("finvol.csv", ',')
tech_vol = DelimitedFiles.readdlm("techvol.csv", ',')

132×359 Array{Float64,2}:
      2.40809e7       2.3876e7   …       2.1874e7        4.18452e7
      2.56219e6       2.17024e6     902931.0             1.9954e6 
      2.36906e6       3.97441e6          1.10139e6       2.12514e6
 271199.0        189537.0            56747.0        154205.0      
      9.76372e6       1.25414e7          5.06396e6       1.07919e7
  18563.0         17905.0        …   35886.0         58494.0      
  76978.0        175290.0            28271.0         55766.0      
 891631.0             1.02075e6     784254.0             1.34333e6
 814367.0             1.21216e6     507616.0        663047.0      
      4.19403e6       7.11424e6          4.99524e6       8.94587e6
      3.10605e6       3.15651e6  …  776416.0             2.45528e6
 829370.0             1.53205e6     698226.0        726995.0      
 201866.0        103638.0            71113.0        380208.0      
      ⋮                          ⋱                                
      9.87231e6       7.84308e6  …  

In [25]:
fin_adv = mean(fin_vol[:,1:200], dims=2)
tech_adv = mean(tech_vol[:,1:200], dims=2)

132×1 Array{Float64,2}:
      2.767241929e7       
      1.9391751e6         
      2.42623198e6        
 400459.055               
      1.1250479165000001e7
  32846.115               
  65294.92                
      1.114116095e6       
 728209.41                
      5.82877133e6        
      3.045857825e6       
 828269.385               
 160990.98500000002       
      ⋮                   
      5.556212845e6       
      4.49222443e6        
 308346.775               
      1.438525745e6       
      1.3075516300000001e6
      3.792593515e6       
  73240.065               
      1.37506275e6        
 234357.01                
   9126.135               
      2.201907965e6       
      2.19282547e6        

### Filtration
Filtration criteria:<br>
<ul>
    <li>Sector neutral: The pair is in the same sector.</li>
    <li>Beta neutral: The beta (both market beta and index beta) difference between the pair is below 0.2.</li>
    <li>Market cap neutral: the market cap of the pair is comparable with each other, estimated by ADV*price. The smaller one is above 80% of the larger one.</li>
    <li>Liquidity: The stocks are liquid in the market. ADV is above 5 million.</li>
</ul>

In [26]:
fin_pair = []
for i in 1:size(fin_ret, 1)
    for j in i+1:size(fin_ret, 1)
        if abs(fin_beta_spy[i] - fin_beta_spy[j]) < 0.2 && abs(fin_beta_xlf[i] - fin_beta_xlf[j]) < 0.2 && 
            0.8 < (fin_data[i,1]*fin_adv[i])/(fin_data[j,1]*fin_adv[j]) < 1.25 && 
            fin_adv[i] > 5.0e6 && fin_adv[j] > 5.0e6
            push!(fin_pair, [i, j])
        end
    end
end

In [27]:
fin_pair

8-element Array{Any,1}:
 [45, 222] 
 [85, 237] 
 [85, 452] 
 [237, 452]
 [240, 357]
 [240, 399]
 [271, 438]
 [357, 399]

In [28]:
fin_ticker = DelimitedFiles.readdlm("finticker.csv", ',')
fin_ticker[[45, 85, 222, 237, 240, 271, 357, 399, 438, 452]]

10-element Array{Any,1}:
 "BBD"
 "C"  
 "IBN"
 "JPM"
 "KEY"
 "MET"
 "RF" 
 "SYF"
 "USB"
 "WFC"

In [29]:
tech_pair = []
for i in 1:size(tech_ret, 1)
    for j in i+1:size(tech_ret, 1)
        if abs(tech_beta_spy[i] - tech_beta_spy[j]) < 0.2 && abs(tech_beta_xlk[i] - tech_beta_xlk[j]) < 0.2 &&
            0.8 < (tech_data[i,1]*tech_adv[i])/(tech_data[j,1]*tech_adv[j]) < 1.25 &&
            tech_adv[i] > 5.0e6 && tech_adv[j] > 5.0e6
            push!(tech_pair, [i, j])
        end
    end
end

In [30]:
tech_pair

8-element Array{Any,1}:
 [29, 100] 
 [47, 51]  
 [47, 52]  
 [47, 121] 
 [51, 52]  
 [51, 121] 
 [52, 121] 
 [115, 121]

In [31]:
tech_ticker = DelimitedFiles.readdlm("techticker.csv", ',')
tech_ticker[[29, 47, 51, 52, 100, 115, 121]]

7-element Array{Any,1}:
 "CSCO"
 "GLW" 
 "HPE" 
 "HPQ" 
 "QCOM"
 "SYMC"
 "TSM" 